In [19]:
import tensorflow as tf
from flows import NormalRW, DFlow, ResFlow, NVPFlow, phase, GVAR, Normal, LogNormal, floatX, MVNormal, MVNormalRW, Linear, CFlow, LinearChol
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from tensorflow.contrib.distributions import WishartCholesky

np.random.seed(1234)
tf.set_random_seed(1234)

In [2]:
data = pd.read_csv('./CDATA/FRA.csv', index_col='VARIABLE').values.astype(floatX).T[np.newaxis][:,:,:3]

In [3]:
data.shape

(1, 195, 3)

In [4]:
stds = (data[0,1:] - data[0,:-1]).std(axis=0)
print(stds)
data /= stds

[0.0057413  0.92696436 0.00503917]


In [5]:
xs = data

In [6]:
n = data.shape[1]
s1 = 0.007
s0 = 2.
dim = [3,4]

k = dim[0] * dim[1]
with tf.variable_scope('pwalk_inf', reuse=tf.AUTO_REUSE):
    ldiag = DFlow([NVPFlow(dim=k, name='ldiag_flow_' + str(i)) for i in range(4)])
    ldiag.output += np.log(1/s1)
    ldiag.logdens -= tf.reduce_sum(ldiag.output)

PWalk = MVNormalRW(dim[0]*dim[1], sigma=s1, sigma0=s0, name='param_walk_prior', 
                   ldiag=ldiag.output[0])
# PWalk = MVNormalRW(dim[0]*dim[1], sigma=s1, sigma0=s0, name='param_walk_prior')
# PWalk = NormalRW(dim=None, sigma0=s0, sigma=s1)

In [7]:
# ldiag.logdens

In [8]:
def autoregr_tf(X, param):
    d = param[:,:dim[-1]-1]
    X = tf.matmul(X, d) + param[:,-1][np.newaxis]
    return X

In [9]:
out_model = GVAR(dim[0]*dim[1], len=xs.shape[1], name='GVAR')

In [10]:
outputs = out_model.sample()

In [11]:
outputs

<tf.Tensor 'GVAR_1/VAR/strided_slice:0' shape=(1, 195, 12) dtype=float64>

In [12]:
prior = PWalk.logdens(outputs, reduce=False)

In [14]:
prior = tf.reduce_sum(prior)

In [15]:
def create_loglik2():
    obs_d = Normal(dim=None, sigma=1., mu=0)
    out = tf.reshape(outputs, [n, dim[0], dim[1]])
    
    def step1(current):
        X = current[0][np.newaxis]
        param = current[1]
        d = param[:,:dim[-1]-1]
        X = tf.matmul(X, d)[0] + param[:,-1]
        return X
    
    ar = tf.map_fn(step1, (xs[0][:-1], out[:-1]), dtype=tf.float64)
    return obs_d.logdens(ar - xs[0][1:])

In [16]:
logl = create_loglik2()

In [17]:
ent = out_model.logdens

In [18]:
ent

<tf.Tensor 'GVAR_1/VAR/logdens/add:0' shape=() dtype=float64>

In [20]:
# df = dim[0]*dim[1]
# pmat = np.diag([(1/s1**2)]*(dim[0]*dim[1]))/df
# cov_prior = WishartCholesky(df, pmat, cholesky_input_output_matrices=True)
import math
cov_prior = LogNormal(None, mu=-0.5*math.log(s1), sigma=1.5)

In [21]:
prior

<tf.Tensor 'Sum:0' shape=() dtype=float64>

In [22]:
loss = -logl - prior + ent - cov_prior.logdens(ldiag.output) + ldiag.logdens# + lowerd.logdens- cov_prior.log_prob(PWalk.fsigma)
loss /= n*dim[0]*dim[1]

In [23]:
tf.summary.scalar('loss', loss[0])

<tf.Tensor 'loss:0' shape=() dtype=string>

In [24]:
with tf.variable_scope('opt') as scope:
    opt = tf.train.AdamOptimizer(0.0001).minimize(loss)

In [25]:
graph = tf.get_default_graph()

In [26]:
opt_vs = graph.get_collection('variables', scope=scope.name)

In [27]:
graph.get_all_collection_keys()

['variables', 'trainable_variables', 'while_context', 'summaries', 'train_op']

In [28]:
# tf.variables_initializer(opt_vs).run()

In [29]:
sess = tf.InteractiveSession()

In [30]:
tf.global_variables_initializer().run()

In [31]:
prior.eval(), ent.eval(), logl.eval()

(2504.5704283581767, 7493.5280618367115, -2426.170679166925)

In [32]:
loss.eval()

array([6.87293439])

In [33]:
np.set_printoptions(precision=4)

In [34]:
# PWalk.inverse_sigma.eval()[:3,:3]

In [35]:
# !rm -R /tmp/tfdbg
writer = tf.summary.FileWriter('/tmp/tfdbg/modelfra_nonvp_correct_mvnormal_xTl_entfix_0.0001_nvp_soft_p')

In [36]:
tf.summary.scalar('s1', tf.reduce_mean(tf.sqrt(tf.diag_part(PWalk.sigma))))

<tf.Tensor 's1:0' shape=() dtype=string>

In [37]:
all_sum = tf.summary.merge_all()

In [38]:
out = tf.reshape(outputs, [n, dim[0], dim[1]])

In [39]:
epoch = 0

In [40]:
# for epoch in range(epoch, 20):
#     for i in range(800):
#         l, _ = sess.run([loss, opt], {phase:False})
#     print(l)

In [41]:
%%time
for epoch in range(epoch, epoch+10000):
    for i in range(100):
        l, _ = sess.run([loss, opt], {phase:True})
        if i % 10 == 0 and epoch > 2:
            s = all_sum.eval({phase:True})
            writer.add_summary(s, global_step=epoch*100 + i)
    print(l)
#     print(PWalk.inverse_sigma.eval()[:3,:3])
#     print('\n')

[3.9521]
[4.6909]
[4.0678]
[125.8488]
[7.7795]
[6.8401]
[6.1231]
[1.9669]
[4.0734]
[3.8925]
[2.1006]
[3.2009]
[2.0923]
[4.3856]
[2.4641]
[2.1135]
[2.5913]
[4.4389]
[2.8212]
[2.3895]
[2.3852]
[1.7255]
[3.0829]
[5.4286]
[5.5868]
[6.165]
[17.6904]
[1.7472]
[3.2798]
[2.0944]
[2.5239]
[2.274]
[2.8835]
[1.9303]
[2.1535]
[3.183]
[1.5614]
[2.2267]
[1.7312]
[1.5845]
[1.8622]
[1.7236]
[1.5959]
[1.419]
[1.7174]
[1.7698]
[2.5873]
[1.6427]
[1.663]
[1.5761]
[1.6592]
[1.5946]
[1.5587]
[1.4684]
[1.5504]
[1.423]
[1.5081]
[1.4452]
[1.4817]
[1.5629]
[1.4621]
[1.5222]
[1.4062]
[1.5134]
[1.4956]
[1.4758]
[1.3948]
[1.3712]
[1.4296]
[1.4178]
[1.3897]
[1.498]
[1.463]
[1.6027]
[1.5203]
[1.3967]
[1.4053]
[1.497]
[1.376]
[1.3786]
[1.3356]
[1.3197]
[1.3652]
[1.3401]
[1.3504]
[1.3967]
[1.3467]
[1.4677]
[1.3387]
[1.3682]
[1.3265]
[1.3031]
[1.3041]
[1.3514]
[1.3059]
[1.2905]
[1.2861]
[1.3169]
[1.3615]
[1.3011]
[1.2901]
[1.334]
[1.306]
[1.2927]
[1.2887]
[1.2848]
[1.2957]
[1.2905]
[1.2714]
[1.2959]
[1.2995]
[1.2893]
[

KeyboardInterrupt: 

In [59]:
sd.eval()

array([0.0098, 0.0094, 0.0097, 0.0091, 0.0104, 0.0107, 0.0095, 0.0095,
       0.0117, 0.0116, 0.0096, 0.0102])

In [54]:
sd = tf.sqrt(tf.diag_part(PWalk.sigma))

In [ ]:
np.diag(PWalk.fsigma.eval())

In [ ]:
epoch

In [42]:
ss = []
aa = []
for _ in range(3000):
    a = PWalk.sigma.eval()
    ss.append(np.sqrt(np.diag(a)))
    aa.append(a)
ss = np.array(ss)
aa = np.array(aa)

In [43]:
aa.mean(axis=0)[:5,:5]

array([[ 9.4475e-05, -3.8617e-08,  5.7549e-07,  3.0892e-08,  3.4678e-08],
       [-3.8617e-08,  8.9602e-05,  1.0797e-08,  1.0008e-08,  4.8208e-08],
       [ 5.7549e-07,  1.0797e-08,  9.6222e-05,  3.0806e-07,  1.1184e-07],
       [ 3.0892e-08,  1.0008e-08,  3.0806e-07,  9.4988e-05,  5.2669e-08],
       [ 3.4678e-08,  4.8208e-08,  1.1184e-07,  5.2669e-08,  9.9481e-05]])

In [44]:
np.mean(ss, axis=0), np.std(ss, axis=0)

(array([0.0097, 0.0095, 0.0098, 0.0097, 0.01  , 0.0102, 0.01  , 0.01  ,
        0.0101, 0.0096, 0.0098, 0.0099]),
 array([0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005,
        0.0005, 0.0005, 0.0005, 0.0005]))

In [ ]:
sns.kdeplot(ss[:,1], ss[:,0])
plt.show()

In [46]:
samples = []
for _ in range(8000):
    s = out.eval({phase:True})
    samples.append(s)
samples = np.array(samples)

In [51]:
ix = [2,0]
for d in range(1,20):
    d *= 4
    sh = [-1, 1]
    s1 = np.reshape(samples[:,:-d,ix[0], ix[1]], sh)
    s2 = np.reshape(samples[:,d:,ix[0], ix[1]], sh)
    s1 -= s1.mean()
    s2 -= s2.mean()
    cov = np.mean(s1*s2)
    
    print(d//4, cov/(np.std(s1)*np.std(s2)), np.std(s1))

1 0.9853585627969467 0.11951449908923241
2 0.9699330207748809 0.11855940369785078
3 0.9540701421769443 0.11753452642376816
4 0.9380486971030598 0.11644603560853832
5 0.9214991200851105 0.11530480404434715
6 0.9046759742730607 0.11411441944088356
7 0.8873157053685081 0.11285140446692461
8 0.8696008700402196 0.11151759315037375
9 0.8517242177592718 0.11011580789943186
10 0.8335347300525258 0.10868159891497789
11 0.815021592260705 0.10718593881242232
12 0.7964184828141888 0.10564211548529344
13 0.7776522323781311 0.10409038334626515
14 0.7583533425252832 0.10252306957395807
15 0.7389536515989993 0.10087726478971319
16 0.7194542053223992 0.09919002783475334
17 0.6996552942773816 0.09746879355826221
18 0.680034615287693 0.09569018130113269
19 0.6606821947871078 0.09383871211545233


In [47]:
ix = [2,0]
for d in range(1,20):
    d *= 4
    sh = [-1, 1]
    s1 = np.reshape(samples[:,:-d,ix[0], ix[1]], sh)
    s2 = np.reshape(samples[:,d:,ix[0], ix[1]], sh)
    s1 -= s1.mean()
    s2 -= s2.mean()
    cov = np.mean(s1*s2)
    
    print(d//4, cov/(np.std(s1)*np.std(s2)), np.std(s1))

1 0.9845679092483259 0.11456496910120868
2 0.9686155215778742 0.11362470757932798
3 0.9519230188972749 0.11265143951861485
4 0.934560995321362 0.11163569349824043
5 0.9166273523370817 0.11055196194128945
6 0.898265061165654 0.1093896885526279
7 0.8796232308971386 0.10814681766664433
8 0.8607502841603968 0.10683554444732167
9 0.8418018791693167 0.10543187911072659
10 0.8226648876349049 0.10393617104054294
11 0.8033718254863436 0.102381332550622
12 0.7837755397510248 0.10082583247362373
13 0.7641139228559936 0.0992334502516869
14 0.7444996610407795 0.0975837932735159
15 0.7248335590415698 0.09590255279773185
16 0.7054423792866007 0.09415962841891667
17 0.686251805119828 0.09237664225567412
18 0.667408640055795 0.09057219806544842
19 0.6484978665501832 0.08880998592291706


In [ ]:
t = -1
sns.kdeplot(samples[:,t,2,0], samples[:,0,2,0])
plt.show()

In [ ]:
t = 10
# params[t]

In [ ]:
np.mean(samples[:,t], axis=0)

In [ ]:
np.mean(samples[:,t], axis=0)

In [ ]:
from sklearn.neighbors.kde import KernelDensity
def score(s1, s2):
    bw = 0.1
    s1 = s1[:,np.newaxis]
    s2 = s2[:,np.newaxis]
    e1 = KernelDensity(bandwidth=bw).fit(s1)
    e2 = KernelDensity(bandwidth=bw).fit(s2)
    
    e1e2 = e1.score(s1) - e2.score(s1)
    e2e1 = e2.score(s2) - e1.score(s2)
    dist = (e1e2/len(s1) + e2e1/len(s2))/2
    return dist

In [ ]:
def get_cdf(ts):
    return lambda x: (ts <= x).mean()
def score(s1,s2):
    c1 = get_cdf(s1)
    c2 = get_cdf(s2)

    m = 0
    for s in np.concatenate([s1, s2]):
        r = np.abs(c1(s) - c2(s))
        if r > m:
            m = r
    return m

In [ ]:
score(s1,s2)

In [ ]:
dists = []
for t1 in range(samples.shape[1] - 1):
    np.random.seed(1234)
    print(t1)
    t2 = t1 + 1
    s1 = samples[:,t1,0,0]
    s2 = samples[:,t2,0,0]
    ss = []
    for _ in range(10):
        s1_ = np.random.choice(s1, size=8000)
        s2_ = np.random.choice(s2, size=8000)
        m = score(s1_,s2_)
        ss.append(m)
    dists.append(ss)
dists = np.array(dists).T

## KDE + KL

In [ ]:
sns.tsplot(dists, ci=[50, 95])
plt.show()

In [ ]:
sns.tsplot(dists, ci=[50, 95])
plt.show()

## Kolmogorov-Smirnov

In [ ]:
sns.tsplot(dists, ci=[50, 95])
plt.show()

In [ ]:
tf.pow()